In [8]:
import numpy as np
P = np.array([
    [0.1, 0.4, 0.5],
    [0.3, 0.2, 0.5],
    [0.6, 0.35, 0.05]
    ])


In [13]:
k = int(input("Введите кол-во шагов\n ->"))
result = P
for i in range(k-1):
    result = result @ P

print(result)

Введите кол-во шагов
 ->2
[[0.43   0.295  0.275 ]
 [0.39   0.335  0.275 ]
 [0.195  0.3275 0.4775]]


In [19]:
A = np.array([[0.1, 0.2, 0.3]])
k = int(input("Введите кол-во шагов\n ->"))
for i in range(k):
    A = A @ P

print(A)

Введите кол-во шагов
 ->200
[[0.20172414 0.19137931 0.20689655]]
